## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for ICD-10-PCS

- **Model**: `icd10pcs_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts clinical entities from clinical text and maps them to their corresponding ICD-10-PCS codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "icd10pcs-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    "A 45-year-old female patient presented with persistent chest pain and shortness of breath. After a thorough evaluation, it was decided to perform a coronary artery bypass graft (CABG) surgery. Post-operatively, the patient will be monitored in the intensive care unit (ICU) and will undergo cardiac rehabilitation.", 
    "A 30-year-old male patient arrived with severe trauma from a car accident. An emergency laparotomy was performed to control internal bleeding. The surgical team also carried out a splenectomy due to a ruptured spleen. The patient was stabilized and transferred to the surgical intensive care unit (SICU) for further observation and care.",
]



sample_text = """Given the severity of her abdominal examination and her persistence of her symptoms, it is detected that need for laparoscopic appendectomy and possible jejunectomy as well as pyeloplasty. We recommend performing a mediastinoscopy"""

### JSON

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,concept_name_detailed,all_resolutions,all_score
0,114,138,laparoscopic appendectomy,Procedure,0.80225,0DBJ8ZZ,"excision of appendix, endo",0.756066,"[0DBJ8ZZ, 0DTJ8ZZ, 0DBJ0ZZ, 0DTJ0ZZ, 0DBJ4ZX]","[excision of appendix, endo [excision of appendix, endo], resection of appendix, endo [resection of appendix, endo], excision of appendix, open approach [excision of appendix, open approach], resection of appendix, open approach [resection of appendix, open approach], excision of appendix, perc endo approach, diagn [excision of appendix, perc endo approach, diagn]]","[excision of appendix, endo, resection of appendix, endo, excision of appendix, open approach, resection of appendix, open approach, excision of appendix, perc endo approach, diagn]","[0.7560662627220154, 0.7540384531021118, 0.7539170980453491, 0.7508572340011597, 0.741325855255127]"
1,153,163,jejunectomy,Procedure,0.9984,0DBA8ZZ,"excision of jejunum, endo",0.847592,"[0DBA8ZZ, 0DTA8ZZ, 0DBA0ZZ, 0DTA0ZZ, 0DBA3ZZ]","[excision of jejunum, endo [excision of jejunum, endo], resection of jejunum, endo [resection of jejunum, endo], excision of jejunum, open approach [excision of jejunum, open approach], resection of jejunum, open approach [resection of jejunum, open approach], excision of jejunum, percutaneous approach [excision of jejunum, percutaneous approach]]","[excision of jejunum, endo, resection of jejunum, endo, excision of jejunum, open approach, resection of jejunum, open approach, excision of jejunum, percutaneous approach]","[0.8475918769836426, 0.8455489277839661, 0.8215798139572144, 0.8211894035339355, 0.8103318214416504]"
2,176,186,pyeloplasty,Procedure,0.9992,0TV70ZZ,"restriction of left ureter, open approach",0.741653,"[0TV70ZZ, 0T14477, 0T13477, 0TQ43ZZ, 0T14476]","[restriction of left ureter, open approach [restriction of left ureter, open approach], bypass left kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach [bypass left kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach], bypass right kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach [bypass right kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach], repair left kidney pelvis, percutaneous approach [repair left kidney pelvis, percutaneous approach], bypass left kidney pelvis to right ureter with autologous tissue substitute, percutaneous endoscopic approach [bypass left kidney pelvis to right ureter with autologous tissue substitute, percutaneous endoscopic approach]]","[restriction of left ureter, open approach, bypass left kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach, bypass right kidney pelvis to left ureter with autologous tissue substitute, percutaneous endoscopic approach, repair left kidney pelvis, percutaneous approach, bypass left kidney pelvis to right ureter with autologous tissue substitute, percutaneous endoscopic approach]","[0.7416528463363647, 0.7383493185043335, 0.7379072904586792, 0.7371262311935425, 0.736879825592041]"
3,215,229,mediastinoscopy,Procedure,0.9703,0WJC4ZZ,"inspection of mediastinum, percutaneous endoscopic approach",0.811230,"[0WJC4ZZ, 0WBC4ZX, 0WJC0ZZ, 0WJC3ZZ, 0WBC4ZZ]","[inspection of mediastinum, percutaneous endoscopic approach [inspection of mediastinum, percutaneous endoscopic approach], excision of mediastinum, percutaneous endoscopic approach, diagnostic [excision of mediastinum, percutaneous endoscopic approach, diagnostic], inspection of mediastinum, open approach [inspection of mediastinum, open approach], inspection of mediastinum, percutaneous approach [inspection of mediastinum, percutaneous approach], excision of mediastinum, percutaneous endoscopic approach [excision of mediastinum, percutaneous endoscopic approach]]","[inspection of medi

### JSON Lines

In [11]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [12]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 114, "end": 138, "ner_chunk": "laparoscopic appendectomy", "ner_label": "Procedure", "ner_confidence": "0.80225", "concept_code": "0DBJ8ZZ", "resolution": "excision of appendix, endo", "score": 0.7560662627220154, "all_codes": ["0DBJ8ZZ", "0DTJ8ZZ", "0DBJ0ZZ", "0DTJ0ZZ", "0DBJ4ZX"], "concept_name_detailed": ["excision of appendix, endo [excision of appendix, endo]", "resection of appendix, endo [resection of appendix, endo]", "excision of appendix, open approach [excision of appendix, open approach]", "resection of appendix, open approach [resection of appendix, open approach]", "excision of appendix, perc endo approach, diagn [excision of appendix, perc endo approach, diagn]"], "all_resolutions": ["excision of appendix, endo", "resection of appendix, endo", "excision of appendix, open approach", "resection of appendix, open approach", "excision of appendix, perc endo approach, diagn"], "all_score": [0.7560662627220154, 0.7540384531021118, 0.7539170980453491,

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [14]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [15]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [16]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [21]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 146,
    'end': 175,
    'ner_chunk': 'a coronary artery bypass graft',
    'ner_label': 'Procedure',
    'ner_confidence': '0.81618005',
    'concept_code': '02100J3',
    'resolution': 'bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach',
    'score': 0.8115347623825073,
    'all_codes': ['02100J3', '02100JW', '02100J9', '02100KW', '02100JC'],
    'concept_name_detailed': ['bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach [bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach]',
     'bypass coronary artery, one artery from aorta with synthetic substitute, open approach [bypass coronary artery, one artery from aorta with synthetic substitute, open approach]',
     'bypass coronary artery, one artery from left internal mammary with synthetic substitute, open approach [bypass coronary artery, one artery from left internal m

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [24]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 146, "end": 175, "ner_chunk": "a coronary artery bypass graft", "ner_label": "Procedure", "ner_confidence": "0.81618005", "concept_code": "02100J3", "resolution": "bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach", "score": 0.8115347623825073, "all_codes": ["02100J3", "02100JW", "02100J9", "02100KW", "02100JC"], "concept_name_detailed": ["bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach [bypass coronary artery, one artery from coronary artery with synthetic substitute, open approach]", "bypass coronary artery, one artery from aorta with synthetic substitute, open approach [bypass coronary artery, one artery from aorta with synthetic substitute, open approach]", "bypass coronary artery, one artery from left internal mammary with synthetic substitute, open approach [bypass coronary artery, one artery from left internal mammary with synthetic substitute, open approach

In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: icd10pcs-vdb-resolver-2025-07-31-07-56-28-520


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

